In [ ]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

batch_size = 128
epochs = 10
seed = 1
log_interval = 10
torch.manual_seed(seed)
device = "cuda"

In [ ]:
from Tars.distributions import GaussianModel, BernoulliModel
from Tars.models import VAE

In [ ]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
x_dim = 784
y_dim = 10
z_dim = 64

# inference model q(z|x,y)
class Inference(GaussianModel):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["x","y"], var=["z"])

        self.fc1 = nn.Linear(x_dim+y_dim, 400)
        self.fc21 = nn.Linear(400, z_dim)
        self.fc22 = nn.Linear(400, z_dim)

    def forward(self, x, y):
        h1 = F.relu(self.fc1(torch.cat([x, y], 1)))
        return self.fc21(h1), F.softplus(self.fc22(h1))
    
# generative model p(x|z,y)    
class Generator(BernoulliModel):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z","y"], var=["x"])

        self.fc3 = nn.Linear(z_dim+y_dim, 400)
        self.fc4 = nn.Linear(400, x_dim)

    def forward(self, z, y):
        h3 = F.relu(self.fc3(torch.cat([z, y], 1)))
        return F.sigmoid(self.fc4(h3))
    
# prior model p(z)
loc = torch.tensor(0.).cuda()
scale = torch.tensor(1.).cuda()
prior = GaussianModel(loc=loc, scale=scale, var=["z"], dim=z_dim)

In [ ]:
p = Generator()
q = Inference()

p.cuda()
q.cuda()

In [ ]:
model = VAE(q, p, prior, optim.Adam, {"lr":1e-3})

In [ ]:
def train(epoch):
    train_loss = 0
    for batch_idx, (data_x, data_y) in enumerate(train_loader):
        data_x = data_x.to(device)
        data_y = torch.eye(10)[data_y].to(device)
        lower_bound, loss = model.train({"x": data_x.view(-1, 784), "y": data_y})
        train_loss += loss
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data_x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test(epoch):
    test_loss = 0
    for i, (data_x, data_y) in enumerate(test_loader):
        data_x = data_x.to(device)
        data_y = torch.eye(10)[data_y].to(device)
        lower_bound, loss = model.test({"x": data_x.view(-1, 784), "y": data_y})
        test_loss += loss
        z = q.sample({"x": data_x.view(-1, 784), "y": data_y})
        recon_batch = p.sample_mean(z)
        if i == 0:
            n = min(data_x.size(0), 8)
            comparison = torch.cat([data_x[:n],
                                  recon_batch.view(batch_size, 1, 28, 28)[:n]])
            save_image(comparison.cpu(),
                     'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        plot_number = 4
        sample_z = 0.3 * torch.randn(64, z_dim).to(device)
        sample_y = torch.eye(10)[[plot_number]*64].to(device)
        
        sample = p.sample_mean({"z":sample_z, "y":sample_y}).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')